In [74]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import cv2

from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

In [46]:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [47]:
tf.__version__


'2.7.0'

In [48]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [49]:
path = "dataset_sidi_v1.2.csv"
df512 = pd.read_csv(path, sep='\t')
df512

,Unnamed: 0,image_name,NEG,NEU,POS,text,sent_text,sent_image,image_path,image_height,image_width
0,0,768097808037605376-1,0.919993,0.055681,0.024326,"RT @hoseokahhh: ""who is this song forwhat do i...",NEG,0,data/76809/768097808037605376-1.jpg,449,449
1,1,768097833215938560-1,0.864726,0.058440,0.076834,RT @ARapperSaid_: “I ain’t come over here for ...,NEG,0,data/76809/768097833215938560-1.jpg,398,593
2,2,768097929659817984-1,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-1.jpg,489,1132
3,3,768097929659817984-2,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-2.jpg,983,1123
4,4,768097929659817984-3,0.907574,0.061396,0.031029,RT @blackedfriction: i hate telling people i l...,NEG,0,data/76809/768097929659817984-3.jpg,1326,1110
...,...,...,...,...,...,...,...,...,...,...,...
470581,470581,804618699315539969-1,0.453963,0.387611,0.158427,Chances not to loose&gt;&gt;&gt;https://t.co/j...,NEG,0,data/80461/804618699315539969-1.jpg,183,275
470582,470582,804618854466981888-1,0.651498,0.208690,0.139812,Don't hear this much RT @phillynewsnow: Lyft ...,NEG,0,data/80461/804618854466981888-1.jpg,675,1200
470583,470583,804619265546387457-1,0.473848,0.382078,0.144074,Security agencies threaten to assassinate me –...,NEG,0,data/80461/804619265546387457-1.jpg,682,1024
470584,470584,804619315861393408-1,0.402850,0.337901,0.259249,"Kill my trusted old assistant? Why, no. I'm go...",NEG,0,data/80461/804619315861393408-1.jpg,458,626


In [50]:
width = 128
height = 128

In [51]:
filenames = ['./Dataset/b-t4sa_imgs/' + fname for fname in df512['image_path'].tolist()]
labels = df512['sent_image'].tolist()
train_filenames, val_filenames, train_labels, val_labels = train_test_split(filenames,labels, train_size=0.9,random_state=42)

In [52]:
from keras.utils.np_utils import to_categorical   

train_labels = to_categorical(train_labels, num_classes=3)
test_labels = to_categorical(val_labels, num_classes=3)

In [53]:
tf.keras.backend.clear_session()
import gc
gc.collect()

394

## CNN básica


In [115]:
tf.keras.backend.clear_session()
import gc
gc.collect()

2644

In [116]:
camadas = 32
model = models.Sequential()
model.add(layers.Conv2D(camadas, (3, 3), activation='relu', input_shape=(width, height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(camadas*2, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(camadas*4, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(camadas*8, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

In [117]:
model.compile(
    optimizer='SGD',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

### FIT GENERATOR

In [118]:
#busca os dados que estão listados em train_filenames

def generate_data(train_filenames,train_labels, batch_size):

    width = 128
    height = 128
    dim = (width, height)
    while True:
        for i in range(0,len(train_filenames),batch_size):
            
            X_train = np.zeros((int(batch_size), width, height, 3))
            aux =0

            dim = (width, height)

            for j in train_filenames[i:i+batch_size]:
                X_train[aux] =  np.asarray(cv2.resize(cv2.imread(j), dim, interpolation = cv2.INTER_AREA) )
                aux = aux + 1
                
            img = X_train.reshape(( X_train.shape[0],) +(width, height,3)).astype('float32') / 255.
            img = preprocess_input(img) 
            
            yield img, np.array(test_labels[i:i+batch_size])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=1,  restore_best_weights=True)

batch_size=32


model.fit( generate_data(train_filenames,train_labels, batch_size),
        batch_size=batch_size,  epochs=5)

Epoch 1/5
    581/Unknown - 392s 675ms/step - loss: 3.6643 - accuracy: 0.3336

### Resultados

In [72]:
x_test = np.zeros((int(100), width, height, 3))
aux =0

dim = (width, height)

for i in val_filenames[:100]:
    x_test[aux] = np.asarray(cv2.resize(cv2.imread(i), dim, interpolation = cv2.INTER_AREA) )
    aux = aux + 1
x_test = x_test.reshape((x_test.shape[0],) +(width, height,3)).astype('float32') / 255.

In [71]:
del x_test

In [76]:
y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=-1)
test_labels_result= np.argmax(test_labels[:100], axis=-1)

print(accuracy_score(test_labels_result, y_pred));
print(classification_report(test_labels_result, y_pred));

0.3
              precision    recall  f1-score   support

           0       0.31      0.90      0.46        31
           1       0.00      0.00      0.00        29
           2       0.22      0.05      0.08        40

    accuracy                           0.30       100
   macro avg       0.18      0.32      0.18       100
weighted avg       0.18      0.30      0.17       100



C:\Users\bhla\anaconda3\envs\DP\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
